In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.stats import poisson
import statistics as st
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.6.0


In [3]:
upcoming_caf = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/UEFA/INPUT/UAFA_upcoming.csv")
caf = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/UEFA/INPUT/UAFA_rank_input.csv")
# home_xg = conmebol["H_xG"]
# away_xg = conmebol["A_xG"]

In [4]:
# Regression model - Neural network for home_xG, away_xg
home_xg_df = caf[['home_score', 'H_attack', 'A_defence']]
away_xg_df = caf[['away_score', 'H_defence', 'A_attack']]

# Splitting dataset
train_homexG = home_xg_df.sample(frac=0.8, random_state = 0)
test_homexG = home_xg_df.drop(train_homexG.index)

train_awayxG = away_xg_df.sample(frac=0.8, random_state = 0)
test_awayxG = away_xg_df.drop(train_awayxG.index)

# For home_xG
train_features = train_homexG.copy()
test_features = test_homexG.copy()

train_labels = train_features.pop('home_score')
test_labels = test_features.pop('home_score')

# For away_xG
train_features2 = train_awayxG.copy()
test_features2 = test_awayxG.copy()

train_labels2 = train_features2.pop('away_score')
test_labels2 = test_features2.pop('away_score')

In [5]:
# Pre-processing, home / away
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

normalizer2 = preprocessing.Normalization(axis=-1)
normalizer2.adapt(np.array(train_features2))

# Model functions, home / away
# home
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])
# away
linear_model2 = tf.keras.Sequential([
    normalizer2,
    layers.Dense(units=1)
])

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

# Building model, home / away
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

dnn_model2 = build_and_compile_model(normalizer2)
dnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 2)                 5         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 4,422
Trainable params: 4,417
Non-trainable params: 5
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 2)                 5         
______________________________

In [6]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=10000)

# training model home

CPU times: user 11min 7s, sys: 1min 59s, total: 13min 7s
Wall time: 7min 55s


In [7]:
%%time
history = dnn_model2.fit(
    train_features2, train_labels2,
    validation_split=0.2,
    verbose=0, epochs=10000)

# training model away

CPU times: user 11min 24s, sys: 2min 1s, total: 13min 26s
Wall time: 7min 59s


In [8]:
test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
pd.DataFrame(test_results, index=['Mean absolute error [home_score]']).T

test_results2 = {}
test_results2['dnn_model2'] = dnn_model2.evaluate(test_features2, test_labels2, verbose=0)
pd.DataFrame(test_results2, index=['Mean absolute error [away_score]']).T


,Mean absolute error [away_score]
dnn_model2,1.003207


In [24]:
# Predicting home_xG and away_xG for upcoming fixtures
home_xg_features = upcoming_caf[['H_attack', 'A_defence']]
pred_home_xg = dnn_model.predict(home_xg_features).flatten()
pred_home_xg = [0 if i < 0 else i for i in pred_home_xg]
pred_home_xg = [4 if i > 4 else i for i in pred_home_xg]
away_xg_features = upcoming_caf[['H_defence', 'A_attack']]
pred_away_xg = dnn_model2.predict(away_xg_features).flatten()
pred_away_xg = [0 if i < 0 else i for i in pred_away_xg]
pred_away_xg = [4 if i > 4 else i for i in pred_away_xg]
pred_away_xg

[0.49876332,
 4,
 4,
 4,
 1.2571987,
 0,
 0,
 0.77136683,
 1.058565,
 0,
 1.8068982,
 0.7460729,
 2.7295003,
 0.4930619,
 0,
 4,
 0.7940403,
 2.3347826,
 1.0008732,
 0.10597265,
 1.104546,
 0.79851973,
 0.9520875,
 0.39256895,
 0.6644329,
 1.1165972,
 0.7021266,
 0.60806787,
 1.2252573,
 0,
 1.4280442,
 0,
 0.22296345,
 0.77752864,
 0.5465481,
 0,
 1.2073063,
 0.76018345,
 1.0941511,
 0.24467123,
 0.88916254,
 0.75817466,
 4,
 4,
 0.85866606,
 0.7602383,
 0.14469004,
 1.1727697,
 1.7977475,
 0.51364625,
 0.7977754,
 0.12960327,
 4,
 0.9597049,
 4,
 1.6959077,
 4,
 0,
 0.14321655,
 0.7580892,
 0.08107048,
 1.1246376,
 4,
 0.3854189,
 0.19137752,
 0.07465291,
 1.2519101,
 1.0186667,
 0.1515336,
 1.2307242,
 0.035557568,
 1.8544828,
 1.0730524,
 1.046195,
 2.724894,
 0.61470973,
 0.9951364,
 4,
 1.8717138,
 0.09689641,
 4,
 2.939838,
 0.97990286,
 0.008536696,
 0.6031971,
 0.81244767,
 1.6196059,
 4,
 1.0959569,
 1.2937745,
 0.90323246,
 0.54851043,
 0,
 2.9936619,
 3.4635286,
 3.4719806,

In [30]:
# Run the poisson results for the home team
home_outcome = []

for i in range(len(pred_home_xg)):
    expected = pred_home_xg[i]
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    home_outcome.append(common)

#home_outcome

In [36]:
# Run the poisson results for the away team
away_outcome = []

for i in range(len(pred_away_xg)):
    expected = pred_away_xg[i]
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    away_outcome.append(common)

away_outcome

[0,
 3,
 4,
 4,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 3,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 4,
 0,
 3,
 1,
 4,
 0,
 0,
 0,
 0,
 1,
 4,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 4,
 1,
 0,
 4,
 2,
 1,
 0,
 0,
 0,
 1,
 4,
 1,
 1,
 0,
 0,
 0,
 3,
 3,
 3,
 0,
 4,
 1,
 0]

In [37]:
upcoming_caf["Poisson_H_xG"] = home_outcome
upcoming_caf["Poisson_A_xG"] = away_outcome
caf_result = upcoming_caf[['Home', 'Away', 'Poisson_H_xG', 'Poisson_A_xG']]

result = []
outcomes = ["H_WIN", "DRAW", "A_WIN"]
for i in range(len(away_outcome)):
    if upcoming_caf["Poisson_H_xG"][i] > upcoming_caf["Poisson_A_xG"][i]:
        result.append(outcomes[0])
    elif upcoming_caf["Poisson_H_xG"][i] == upcoming_caf["Poisson_A_xG"][i]:
        result.append(outcomes[1])
    else:
        result.append(outcomes[2])

caf_result["Results"] = result

H_Points = []
A_Points = []
for i in range(len(away_outcome)):
    if result[i] == "H_WIN":
        H_Points.append(3)
        A_Points.append(0)
    elif result[i] == "DRAW":
        H_Points.append(1)
        A_Points.append(1)
    else:
        H_Points.append(0)
        A_Points.append(3)
       
        
caf_result["H_Points"] = H_Points
caf_result["A_Points"] = A_Points
caf_result

,Home,Away,Poisson_H_xG,Poisson_A_xG,Results,H_Points,A_Points
0,Iceland,Armenia,0,0,DRAW,1,1
1,Malta,Slovenia,1,3,A_WIN,0,3
2,Liechtenstein,North Macedonia,0,4,A_WIN,0,3
3,Gibraltar,Montenegro,1,4,A_WIN,0,3
4,Czech Republic,Wales,0,1,A_WIN,0,3
...,...,...,...,...,...,...,...
95,Czech Republic,Estonia,0,3,A_WIN,0,3
96,Netherlands,Norway,2,0,H_WIN,3,0
97,Gibraltar,Latvia,1,4,A_WIN,0,3
98,Montenegro,Turkey,1,1,DRAW,1,1


In [38]:
# Create Points Table
result_table = pd.DataFrame(columns=['Team','GF','GA', 'Points'])
caf_home = caf_result.groupby("Home")["H_Points"].sum()
caf_away = caf_result.groupby("Away")["A_Points"].sum()
points = caf_home + caf_away
H_GF = caf_result.groupby("Home")["Poisson_H_xG"].sum()
A_GF = caf_result.groupby("Away")["Poisson_A_xG"].sum()
H_GA = caf_result.groupby("Home")["Poisson_A_xG"].sum()
A_GA = caf_result.groupby("Away")["Poisson_H_xG"].sum()

GF = H_GF + A_GF
GA = H_GA + A_GA

team_list = pd.unique(caf_result["Home"])
team_list_sort = sorted(team_list)

result_table["Points"] = points
result_table["GF"] = GF
result_table["GA"] = GA
result_table["Team"] = team_list_sort
result_table

,Team,GF,GA,Points
Home,,,,
Albania,Albania,1,9,0
Andorra,Andorra,4,1,5
Armenia,Armenia,2,8,2
Austria,Austria,2,1,7
Azerbaijan,Azerbaijan,2,4,6
Belarus,Belarus,2,1,5
Belgium,Belgium,6,3,6
Bosnia and Herzegovina,Bosnia and Herzegovina,3,7,2
Bulgaria,Bulgaria,1,3,2


In [ ]:
#caf_result.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/OUTPUT/caf_results.csv", index=False)
#result_table.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/OUTPUT/caf_points_table.csv", index=False)